In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import plotly.express as px
import plotly.graph_objects as go

In [2]:
df_life = pd.read_csv('data/lifehistory_df.csv')
df_basiceco = pd.read_csv('data/basicecodf.csv')
df_species = pd.read_csv("data/species.csv")
df_food = pd.read_csv("data/fooddf.csv")

In [3]:
features = df_life[["am", "Wwi", "Ri", "Wwb", "Li", "ab"]]
X = StandardScaler().fit_transform(features)

X_std = StandardScaler().fit_transform(X)

pca = PCA(n_components=3)
principal_components = pca.fit_transform(X_std)
explained_var = pca.explained_variance_ratio_ * 100

# DataFrame PCA
pca_df = pd.DataFrame(data=principal_components, columns=["PC1", "PC2", "PC3"])
pca_df["species"] = df_life["species"]
final_df=pca_df.copy()

final_df['Wwi'] = df_life['Wwi']
final_df['Ri'] = df_life['Ri']
final_df['ecozone'] = df_basiceco['ecozone']
final_df['gender'] = df_basiceco['gender']
final_df['reproduction'] = df_basiceco['reproduction']
final_df['embryo'] = df_basiceco['embryo']
final_df['food'] = df_food['adult']
final_df['order'] = df_species['Order']  
final_df['class'] = df_species['Class'] 
final_df['phylum'] = df_species['Phylum']

In [4]:

fig = px.scatter_3d(
    pca_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color=final_df['Wwi'],
    hover_name="species",
    color_continuous_scale=px.colors.sequential.Viridis,
    opacity=0.8
)

loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

scale = 3  

for i, feature in enumerate(features):
    fig.add_trace(
        go.Scatter3d(
            x=[0, loadings[i, 0] * scale],
            y=[0, loadings[i, 1] * scale],
            z=[0, loadings[i, 2] * scale],
            mode="lines+text",
            line=dict(color="black", width=4),
            text=[None, feature],
            textposition="top center",
            name=feature,
            showlegend=False
        )
    )

fig.update_layout(
    title="PCA 3D",
    scene=dict(
        xaxis_title=f"PC1 ({explained_var[0]:.1f}%)",
        yaxis_title=f"PC2 ({explained_var[1]:.1f}%)",
        zaxis_title=f"PC3 ({explained_var[2]:.1f}%)",
    ),
)
fig.update_traces(marker=dict(size=3.5))  

fig.show()